The script bellow will scrape data from https://www.billboard.com/charts/billboard-200/ a website that contains the top 200 songs of the week. The data will be stored in a json file.

to run the script, an aditional library is needed, pydantic. To install it, run the following command in your terminal:
~~~bash
$ pip install pydantic
~~~

> Note: Pydantic is a validation library, focused on loading the data into a model that makes sure that is stronlgly typed. I absolutely recommend it. 

In [1]:
import sys
from datetime import datetime
from typing import List

import bs4
from pydantic import BaseModel, HttpUrl, ValidationError
from requests import get

TARGET_URL = 'https://www.billboard.com/charts/billboard-200/'


class BillBoardItem(BaseModel):
    # scraped_timestamp is of type datetime. if not suplied, it will be set by default to datetime.now()
    scraped_timestamp: datetime = datetime.now()
    this_week_rank: int
    # url is of type HttpUrl, which is a str with a valid URL.
    url_photo: HttpUrl
    artist: str
    track_name: str
    last_week_rank: str
    peak_position: int
    weeks_on_chart: str


def get_parser(text) -> bs4.BeautifulSoup:
    parser = bs4.BeautifulSoup(
        markup=text,
        features='html.parser'
    )
    return parser


def parse_result(container_row: bs4.Tag) -> BillBoardItem:
    last_week_rank,peak_position, weeks_on_chart =[t.text.strip() for t in container_row.select('li.o-chart-results-list__item > span.c-label')[-3:]]
    this_week_rank, last_week_compare, track_name, artist_name = [t.text.strip() for t in container_row.select('li.o-chart-results-list__item span.c-label')[:4]]
    image_urls = [img.attrs['data-lazy-src'] for img in container_row.select('img.c-lazy-image__img')]
    track_name = container_row.select_one('#title-of-a-story').text.strip()
    label= container_row.select_one('h3.c-title ~ p.c-tagline').text.strip
    data = {
        'this_week_rank': this_week_rank,
        'url_photo': image_urls[-1],
        'label': label,
        'track_name': track_name,
        'artist': artist_name,
        'last_week_rank': last_week_rank,
        'peak_position': peak_position,
        'weeks_on_chart': weeks_on_chart
        }
    try:
        return BillBoardItem(**data)
    except ValidationError as e:
        print(data)
        raise e
def parse_website(text: str) -> List[BillBoardItem]:
    parser = get_parser(text)
    items: List[BillBoardItem] = []
    for idx, tag in enumerate(parser.find_all('div', class_={'o-chart-results-list-row-container'})):
        items.append(parse_result(container_row=tag))
    return items


def main(**kwargs):
    output = kwargs.get('output', sys.stdout)

    r = get(url=TARGET_URL)
    r.raise_for_status()
    text = r.text
    elements = parse_website(text)
    for e in elements:
        print(e.json(), file=output)


with open('billboard200.jsonl', 'w') as f:
    main(output=f)


### Exercises:
##### Grouping Key: 
Every week the entries are moving (or added/removed) according to their popularity.
Create key based on the data to track their movement over the weeks.
 - Modify the BillBoardItem to include an id field, that auto-populates based on the data
     - and read about validators (https://docs.pydantic.dev/usage/validators/) on how to autopopulate it

##### Add awards to the data:
- Modify the BillBoardItem to include an awards field, that auto-populates based on the data
- tip: start with a sample row, and then try to extract the awards from it. 
    - Use the following code get started.
    - get a list of all the rows, and then use the website to determine which row has awards on it
    - then bit by bit try to extract the awards fields
    - after you're succefull, modify the code to add the awards to the BillBoardItem

In [ ]:
import sys
from datetime import datetime
from typing import List

import bs4
from pydantic import BaseModel, HttpUrl, ValidationError
from requests import get

TARGET_URL = 'https://www.billboard.com/charts/billboard-200/'
def get_parser(text) -> bs4.BeautifulSoup:
    parser = bs4.BeautifulSoup(
        markup=text,
        features='html.parser'
    )
    return parser
r = get(url=TARGET_URL)
r.raise_for_status()
text = r.text
parser = get_parser(text)
rows = parser.find_all('div', class_={'o-chart-results-list-row-container'})
# which row has awards so you can work with it? chekc the site
row = rows[1]